In [1]:
from folium import Map, Polygon as fPolygon, Marker, Icon
from shapely.geometry import Point, Polygon
from shapely.ops import transform
import pandas as pd
import numpy as np
import pyproj
import json

In [2]:
with open('data/subway/subway_station.json', 'r') as f:
    stations = json.load(f)

In [3]:
# 위/경도 & 미터 좌표계 변환
def convert_crs(geometry, from_crs='EPSG:4326', to_crs='EPSG:5179'):
    from_crs_instance = pyproj.CRS(from_crs)
    to_crs_instance = pyproj.CRS(to_crs)
    
    project = pyproj.Transformer.from_crs(from_crs_instance, to_crs_instance, always_xy=True).transform
    transformed_geometry = transform(project, geometry)
    return transformed_geometry
    
def convert_lat_lon(geometry):
    if isinstance(geometry, Point):
        return list(geometry.coords)[0][::-1]
    elif isinstance(geometry, Polygon):
        return list(map(lambda x: x[::-1], list(geometry.exterior.coords)))

In [4]:
def generate_buffer(point, meter=100):
    transformed_point = convert_crs(point, from_crs='EPSG:4326', to_crs='EPSG:5179')
    buffer = transformed_point.buffer(meter)
    transformed_buffer = convert_crs(buffer, from_crs='EPSG:5179', to_crs='EPSG:4326')
    return transformed_buffer

def generate_random_point(polygon, number):
    xmin, ymin, xmax, ymax = polygon.bounds
    
    points = []
    while len(points) < number:
        point = Point(np.random.uniform(xmin, xmax), np.random.uniform(ymin, ymax))
        if polygon.contains(point):
            points.append(point)
    return points

In [5]:
m = Map(location=[37.6418, 126.7], zoom_start=13)

for station in stations:
    point = Point([station['start_lon'], station['start_lat']])
    buffer = generate_buffer(point, 100)
    random_points = generate_random_point(buffer, 10)

    station_direction = station['direction']
    Marker(location=convert_lat_lon(point), icon=Icon(color='red' if station_direction == 1 else 'green')).add_to(m)
    fPolygon(locations=convert_lat_lon(buffer)).add_to(m)

    for point in random_points:
        Marker(location=convert_lat_lon(point)).add_to(m)
        
m

---
## generate random passenger

> 승객 : 10000명 기준  
- 지하철 : 9320명  
- 버스   : 680명  

> 지하철 : 역별 OD 비율  
> 버스 : 정류장별 OD??

In [92]:
person_n = 10000
subway_person_n = 8000
bus_person_n = 680 + 1320

start_time = 6 * 60 + 30 # 6시 30분
end_time = 8 * 60  # 8시

## 1. subway passenger

### 1-1. read subway station rate file

In [93]:
subway_station_rate = pd.read_csv('data/passenger/subway_rate.csv')
subway_station_rate

,정류장 명칭_승차,정류장 명칭_하차,count,rate,direction
0,장기,구래,4019,0.0554,2
1,구래,장기,3855,0.0532,1
2,구래,사우(김포시청),2960,0.0408,1
3,사우(김포시청),구래,2948,0.0407,2
4,사우(김포시청),장기,2870,0.0396,2
...,...,...,...,...,...
85,풍무,양촌,16,0.0002,2
86,운양,양촌,13,0.0002,2
87,양촌,풍무,13,0.0002,1
88,마산,양촌,11,0.0002,2


### 1-2. subway station 재정의

In [94]:
new_stations = []

d1_station_lst = list(filter(lambda x: x['direction'] == 1, stations))
d2_station_lst = list(filter(lambda x: x['direction'] == 2, stations))

for idx, station in enumerate(d1_station_lst):
    new_stations.append({
        'station_name': station['start_station'],
        'lat': station['start_lat'],
        'lon': station['start_lon'],
        'direction': 1,
    })
    
    if idx == len(d1_station_lst) - 1:
        new_stations.append({
            'station_name': station['end_station'],
            'lat': station['end_lat'],
            'lon': station['end_lon'],
            'direction': 1,
        })

for idx, station in enumerate(d2_station_lst):
    new_stations.append({
        'station_name': station['start_station'],
        'lat': station['start_lat'],
        'lon': station['start_lon'],
        'direction': 2,
    })
    
    if idx == len(d1_station_lst) - 1:
        new_stations.append({
            'station_name': station['end_station'],
            'lat': station['end_lat'],
            'lon': station['end_lon'],
            'direction': 2,
        })
        

### 1-3. generate subway passenger

In [95]:
def get_station_point(station_name, direction):
    station = list(filter(lambda x: x['station_name'] == station_name and x['direction'] == direction, new_stations))[0]
    lat, lon = station['lat'], station['lon']
    return lat, lon

In [96]:
subway_station_rate['person_n'] = subway_station_rate['rate'].map(lambda x: x * subway_person_n).astype('int')

subway_random_passenger = []

for idx, row in subway_station_rate.iterrows():
    data = row.to_dict()
    time_lst = np.linspace(start_time, end_time, data['person_n'], dtype=int)
    
    direction = data['direction']
    start_station, end_station = data['정류장 명칭_승차'], data['정류장 명칭_하차']
    start_lat, start_lon = get_station_point(start_station, direction)
    end_lat, end_lon = get_station_point(end_station, direction)
    
    ## create random start point
    start_station_point = Point((start_lon, start_lat))
    buffer_polygon = generate_buffer(start_station_point, meter=100)
    random_start_points = generate_random_point(buffer_polygon, data['person_n'])
    
    ## create randon end point
    end_station_point = Point((end_lon, end_lat))
    buffer_polygon = generate_buffer(end_station_point, meter=100)
    random_end_points = generate_random_point(buffer_polygon, data['person_n'])
    
    for i in range(data['person_n']):
        t = time_lst[i]
        
        random_start_point = random_start_points[i]
        random_start_point = list(random_start_point.coords)[0]
        
        random_end_point = random_end_points[i]
        random_end_point = list(random_end_point.coords)[0]
        
        subway_random_passenger.append({
            'start_time': t,
            'start_station': start_station,
            'start_lat': random_start_point[1],
            'start_lon': random_start_point[0],
            'start_station_lat': start_lat,
            'start_station_lon': start_lon,
            'end_station': end_station,
            'end_lat': random_end_point[1],
            'end_lon': random_end_point[0],
            'end_station_lat': end_lat,
            'end_station_lon': end_lon,
            'direction': direction
        })

subway_random_passenger_df = pd.DataFrame(subway_random_passenger)
display(subway_random_passenger_df)

,start_time,start_station,start_lat,start_lon,start_station_lat,start_station_lon,end_station,end_lat,end_lon,end_station_lat,end_station_lon,direction
0,390,장기,37.643481,126.669164,37.643975,126.669070,구래,37.646175,126.628919,37.645315,126.628743,2
1,390,장기,37.644409,126.669428,37.643975,126.669070,구래,37.645058,126.629680,37.645315,126.628743,2
2,390,장기,37.643766,126.668676,37.643975,126.669070,구래,37.644956,126.628220,37.645315,126.628743,2
3,390,장기,37.644826,126.668825,37.643975,126.669070,구래,37.644916,126.628969,37.645315,126.628743,2
4,390,장기,37.644066,126.669106,37.643975,126.669070,구래,37.645289,126.628679,37.645315,126.628743,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7953,390,풍무,37.612592,126.731530,37.612354,126.732439,양촌,37.641942,126.613669,37.641771,126.614740,2
7954,390,운양,37.653133,126.683454,37.653783,126.683931,양촌,37.642415,126.613970,37.641771,126.614740,2
7955,390,양촌,37.641460,126.613755,37.641645,126.614753,풍무,37.612662,126.731356,37.612431,126.732446,1
7956,390,마산,37.640323,126.643965,37.640560,126.644145,양촌,37.641076,126.615234,37.641771,126.614740,2


In [97]:
subway_random_passenger_df.to_csv('data/passenger/decrease_subway_random_passenger.csv', index=None)

## 2. bus passenger - 버스 증차 전

### 2-1. read bus station rate file

In [98]:
bus_station_rate = pd.read_csv('data/passenger/bus_rate.csv')

### 2-2. bus station 재정의

In [99]:
with open('data/bus/bus_station.json', 'r') as f:
    bus_station = json.load(f)

In [100]:
bus70_station = list(filter(lambda x: x['bus_type'] == '70', bus_station))
d1_bus70_station = list(filter(lambda x: x['direction'] == 1, bus70_station))
d2_bus70_station = list(filter(lambda x: x['direction'] == 2, bus70_station))

In [72]:
def get_bus_station(data):
    stations = []
    
    for idx, station in enumerate(data):
        stations.append({
            'bus_type': station['bus_type'],
            'station_name': station['start_station'],
            'lat': station['start_lat'],
            'lon': station['start_lon'],
            'direction': station['direction'],
        })
    
        if idx == len(data) - 1:
            stations.append({
                'bus_type': station['bus_type'],
                'station_name': station['end_station'],
                'lat': station['end_lat'],
                'lon': station['end_lon'],
                'direction': station['direction'],
            })
    return stations

In [73]:
new_stations_70 = get_bus_station(d1_bus70_station) + get_bus_station(d2_bus70_station)

### 2-3. generate bus passenger

In [74]:
def get_station_point(bus_type, stations, station_name, direction):
    station = list(filter(lambda x: x['station_name'] == station_name and x['direction'] == direction and x['bus_type'] == bus_type, stations))[0]
    lat, lon = station['lat'], station['lon']
    return lat, lon

In [78]:
bus70_station_rate = bus_station_rate.copy()
bus70_station_rate['rate'] = bus70_station_rate['count'].map(lambda x: round(x / bus70_station_rate['count'].sum() * 100, 2)) / 100
bus70_station_rate['bus_type'] = '70'

In [79]:
def total_bus_random_passenger(data, bus_station):
    bus_random_passenger = []

    data['person_n'] = data['rate'].map(lambda x: x * int(bus_person_n)).astype('int')
    for idx, row in data.iterrows():
        data = row.to_dict()
        time_lst = np.linspace(start_time, end_time, data['person_n'], dtype=int)
        
        bus_type = data['bus_type']
        direction = data['direction']
        start_station, end_station = data['정류장 명칭_승차'], data['정류장 명칭_하차']
        start_lat, start_lon = get_station_point(bus_type, bus_station, start_station, direction)
        end_lat, end_lon = get_station_point(bus_type, bus_station, end_station, direction)
        
        ## create random start point
        start_station_point = Point((start_lon, start_lat))
        buffer_polygon = generate_buffer(start_station_point, meter=100)
        random_start_points = generate_random_point(buffer_polygon, data['person_n'])
        
        ## create randon end point
        end_station_point = Point((end_lon, end_lat))
        buffer_polygon = generate_buffer(end_station_point, meter=100)
        random_end_points = generate_random_point(buffer_polygon, data['person_n'])
        
        for i in range(data['person_n']):
            t = time_lst[i]
            
            random_start_point = random_start_points[i]
            random_start_point = list(random_start_point.coords)[0]
            
            random_end_point = random_end_points[i]
            random_end_point = list(random_end_point.coords)[0]
            
            bus_random_passenger.append({
                'bus_type': bus_type,
                'start_time': t,
                'start_station': start_station,
                'start_lat': random_start_point[1],
                'start_lon': random_start_point[0],
                'start_station_lat': start_lat,
                'start_station_lon': start_lon,
                'end_station': end_station,
                'end_lat': random_end_point[1],
                'end_lon': random_end_point[0],
                'end_station_lat': end_lat,
                'end_station_lon': end_lon,
                'direction': direction
            })

    bus_random_passenger_df = pd.DataFrame(bus_random_passenger)
    return bus_random_passenger_df

In [81]:
bus70_random_passenger = total_bus_random_passenger(bus70_station_rate, new_stations_70)
bus70_random_passenger.to_csv('data/passenger/prev_bus_random_passenger.csv', index=None)

### 3. bus passenger - 버스 증차 후

### 3-1. read bus station rate file

In [101]:
bus_station_rate = pd.read_csv('data/passenger/bus_rate.csv')

### 3-2. bus station 재정의

In [102]:
with open('data/bus/bus_station.json', 'r') as f:
    bus_station = json.load(f)

In [103]:
bus70_station = list(filter(lambda x: x['bus_type'] == '70', bus_station))
bus70A_station = list(filter(lambda x: x['bus_type'] == '70A', bus_station))
bus70B_station = list(filter(lambda x: x['bus_type'] == '70B', bus_station))

d1_bus70_station = list(filter(lambda x: x['direction'] == 1, bus70_station))
d2_bus70_station = list(filter(lambda x: x['direction'] == 2, bus70_station))

In [104]:
def get_bus_station(data):
    stations = []
    
    for idx, station in enumerate(data):
        stations.append({
            'bus_type': station['bus_type'],
            'station_name': station['start_station'],
            'lat': station['start_lat'],
            'lon': station['start_lon'],
            'direction': station['direction'],
        })
    
        if idx == len(data) - 1:
            stations.append({
                'bus_type': station['bus_type'],
                'station_name': station['end_station'],
                'lat': station['end_lat'],
                'lon': station['end_lon'],
                'direction': station['direction'],
            })
    return stations

In [105]:
new_stations_70 = get_bus_station(d1_bus70_station) + get_bus_station(d2_bus70_station)
new_stations_70A = get_bus_station(bus70A_station)
new_stations_70B = get_bus_station(bus70B_station)

### 3-3. generate bus passenger

In [106]:
def get_station_point(bus_type, stations, station_name, direction):
    station = list(filter(lambda x: x['station_name'] == station_name and x['direction'] == direction and x['bus_type'] == bus_type, stations))[0]
    lat, lon = station['lat'], station['lon']
    return lat, lon

In [107]:
bus70_station_rate = bus_station_rate.copy()
bus70_station_rate['bus_type'] = '70'

bus70A_station_rate = bus_station_rate[(bus_station_rate['정류장 명칭_승차'] != '고촌역') & (bus_station_rate['정류장 명칭_하차'] != '고촌역') & (bus_station_rate['direction'] == 1)].copy()
bus70A_station_rate['rate'] = bus70A_station_rate['count'].map(lambda x: round(x / bus70A_station_rate['count'].sum() * 100, 2)) / 100
bus70A_station_rate['bus_type'] = '70A'

bus70B_station_rate = bus_station_rate[(bus_station_rate['정류장 명칭_승차'] != '풍무역') & (bus_station_rate['정류장 명칭_하차'] != '풍무역') & (bus_station_rate['direction'] == 1)].copy()
bus70B_station_rate['정류장 명칭_승차'] = bus70B_station_rate['정류장 명칭_승차'].map(lambda x: '사우역.김포고' if '걸포북변역' in x else x)
bus70B_station_rate['정류장 명칭_하차'] = bus70B_station_rate['정류장 명칭_하차'].map(lambda x: '사우역.김포고' if '걸포북변역' in x else x)
bus70B_station_rate['rate'] = bus70B_station_rate['count'].map(lambda x: round(x / bus70B_station_rate['count'].sum() * 100, 2)) / 100
bus70B_station_rate['bus_type'] = '70B'

In [108]:
def total_bus_random_passenger(data, bus_station):
    bus_random_passenger = []

    data['person_n'] = data['rate'].map(lambda x: x * int(bus_person_n / 3)).astype('int')
    for idx, row in data.iterrows():
        data = row.to_dict()
        time_lst = np.linspace(start_time, end_time, data['person_n'], dtype=int)
        
        bus_type = data['bus_type']
        direction = data['direction']
        start_station, end_station = data['정류장 명칭_승차'], data['정류장 명칭_하차']
        start_lat, start_lon = get_station_point(bus_type, bus_station, start_station, direction)
        end_lat, end_lon = get_station_point(bus_type, bus_station, end_station, direction)
        
        ## create random start point
        start_station_point = Point((start_lon, start_lat))
        buffer_polygon = generate_buffer(start_station_point, meter=100)
        random_start_points = generate_random_point(buffer_polygon, data['person_n'])
        
        ## create randon end point
        end_station_point = Point((end_lon, end_lat))
        buffer_polygon = generate_buffer(end_station_point, meter=100)
        random_end_points = generate_random_point(buffer_polygon, data['person_n'])
        
        for i in range(data['person_n']):
            t = time_lst[i]
            
            random_start_point = random_start_points[i]
            random_start_point = list(random_start_point.coords)[0]
            
            random_end_point = random_end_points[i]
            random_end_point = list(random_end_point.coords)[0]
            
            bus_random_passenger.append({
                'bus_type': bus_type,
                'start_time': t,
                'start_station': start_station,
                'start_lat': random_start_point[1],
                'start_lon': random_start_point[0],
                'start_station_lat': start_lat,
                'start_station_lon': start_lon,
                'end_station': end_station,
                'end_lat': random_end_point[1],
                'end_lon': random_end_point[0],
                'end_station_lat': end_lat,
                'end_station_lon': end_lon,
                'direction': direction
            })

    bus_random_passenger_df = pd.DataFrame(bus_random_passenger)
    return bus_random_passenger_df

In [109]:
bus70_random_passenger = total_bus_random_passenger(bus70_station_rate, new_stations_70)
bus70A_random_passenger = total_bus_random_passenger(bus70A_station_rate, new_stations_70A)
bus70B_random_passenger = total_bus_random_passenger(bus70B_station_rate, new_stations_70B)

In [110]:
pd.concat([bus70_random_passenger, bus70A_random_passenger, bus70B_random_passenger], axis=0).to_csv('data/passenger/decrease_bus_random_passenger.csv', index=None)